In [5]:
## Import statements and load data
from __future__ import division
import numpy as np
import scipy
from scipy import io
from scipy import special as special
import scipy.stats as stats
from itertools import chain,combinations
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
"""Loading Data"""
controls = scipy.io.loadmat("data/controls.mat")['controls'] #17, 116, 116
depressed = scipy.io.loadmat("data/depressed.mat")['depressed'] #16, 116, 116

# Graph Metrics in fMRI data

## Metric 1 - Modularity

The optimal community structure is a subdivision of the network into
nonoverlapping groups of nodes in a way that maximizes the number of
within-group edges, and minimizes the number of between-group edges.
The modularity is a statistic that quantifies the degree to which the
network may be subdivided into such clearly delineated groups.

The Louvain algorithm is a fast and accurate community detection
algorithm (at the time of writing):
1. Assign each node to its own community i.
2. Calculate Q if remove node from its community i
3. Calculate Q if add node to every neighboring community j
4. Put node into community that results in highest Q increase. If no increase possible, node stays in its own community i.
5. Apply Steps 1-4 for all nodes in graph repeatedly, until no more Q increase possible.
6. Merge all nodes in one community into a new node. Incoming edge weights are summed.
7. Repeat Steps 1-6 with graph of new nodes. 

The implementation of this algorithm (modularity_louvain_und_sign) can be found in modularity.py, as well as several other algorithms related to modularity. Given a connection matrix, this particular implementation returns the grouped members for each module, as well as the modularity of the network. Here we are only interested in the modularity value.

In [7]:
from modularity import modularity_louvain_und_sign
#loop over each individual's data, and put modularity value into a list
controls_modularity = [modularity_louvain_und_sign(controls[i])[1] for i in range(len(controls))]
depressed_modularity = [modularity_louvain_und_sign(depressed[i])[1] for i in range(len(depressed))]

In [9]:
print("modularities for controls: ", controls_modularity)
print("---")
print("modularities for depressed: ", depressed_modularity)

modularities for controls:  [0.059443680818228331, 0.068157281381724111, 0.036696603441131043, 0.046041445064078305, 0.04818120191563325, 0.075665084821964768, 0.043004237544202503, 0.072371828446451306, 0.061087761643920865, 0.050232720453102571, 0.1049458175864072, 0.064073425712800128, 0.056058346922762098, 0.071353344191112111, 0.077269903160571474, 0.045738446346565008, 0.066551116087541121]
---
modularities for depressed:  [0.093798528040905216, 0.20874386153442956, 0.12986902599520722, 0.07149584611675637, 0.12247370674490467, 0.06125322570416606, 0.097108365704986405, 0.065300239894596213, 0.023415746879014133, 0.066976354165625654, 0.097791794975514851, 0.10527739407118321, 0.068366668314025172, 0.1308631546267856, 0.039603224394632201, 0.081486142423716795]


Now that we have a list of modularities for controls, and a list of modularities for depressed individuals, how do we know whether they are different? Can we just compare the average modularities? We will see how to test for significant difference in the statistics section!

## Metric 2

## Metric 3

# Hypothesis Testing in fMRI data

#### Goal : 

The nobel prize winning physicist once aptly stated that "rigor and clarity are not synonymous". Keeping this in mind the intended emphasis of this notebook is clarity. It *is not* our goal to give a rigorous mathematical presentation of various statistical tests and methods that can be used to test connectivity levels between different brain types;however,it *is* our goal to provide clarity on how one could test whether or not there is a "significant difference" (used here in a very colloqiual sense) between the depressed brains and the not depressed brains. That is not to say the former is not important. We strongly reccomend taking a rigorous class in mathematical statisitcs such as Stat 135. However we choose not to delve into the mathematics here because we feel it is first important to understand ,in a very concrete way, *why* taking such a class could be interesting and useful.

#### Abstraction: 

Often times when trying to solve a particular problem with data IRL it is important to be able to abstract away the problem from the particular to the general. Case and point: If we were the first researchers studying links between connectivity of the brain and depression, googling: "How to tell the difference between depressed brains and not depressed brain" would likely not be very helpful. We are on the cutting edge! If google knew the answer to this question then our funding would likely be in jeopardy. However if we were to abstract the problem away from specific, and enter a query such as: "statistical test to determine whether or not 2 different samples were pulled from the same distribution" then ... we would be in buisness. However knowing to enter such a query requires a certain amount of understanding; in particular  knowing to enter such a query requires us to have the ability to **think probabalistically**. 

In short, the main goal of this notebook is to provide a concrete example of how to think probabilistically and computationally to solve a real-world problem. In particular we want to provide the reader with an example of how we can model data with random variables and then draw inferences from that model.

Part of thinking "computationally"(at least in the context of dealing with data) involves the ability to use computers to get an intuition about the structure of the data. A big part of this intuition comes in the form of visualization. Thus we sill start our analysis by visualizing the data.

In [ ]:
## Visualize data 
depressed = np.random.uniform(2,10,16)
happy = np.random.uniform(5,15,17)

plt.plot(depressed, len(depressed) * [0.5], "x")
plt.plot(happy, len(happy) * [1.5], "+")
plt.axis([0, 20,0,2])
plt.show()

#### Model:

Visually we are able to tell that there is definitiely a difference between the two groups. However if we want to try and make this difference a bit more precise. This is where knowing how to think probabalistically becomes very important.

Our given data is the conectivity statistics for $n$ brains, which we will denote $B_1, \ldots, B_n$. Wherein there is some set of indeces $D$ corresponding to depressed brains, and some set of indeces $H$ corresponding to not depressed (happy) brains. What we want to test is whether or not the connectivity of the depressed brains and happy brains are drawn from the same distribution i.e. $B_1,\ldots,B_n \sim \mathbb{B}$ or if they are drawn from fundamentally different distributions i.e. $B_i \sim \mathbb{D} \ \ \forall i \in D$ and $B_i \sim \mathbb{H} \ \ \forall i \in H$.

#### TODO : Talk about distributions (in case they don't know) 




At this point I really want to emphasize that everything shown above is THE MOST IMPORTANT PART OF THIS NOTEBOOK. Because IRL now would be the time to start some frantic googling: http://bfy.tw/DylF to try and figure out a good way to test the differences in brain. If you are able to do what I have described above, namely think probabilistically and computationally you have picked up the main point of this portion of the presentation. That is not to say that the following portion of the notebook is not important. It is just the analysis that follows presents an outline of some of the take home messages, one would have gleamed after a lot of time thinking about the problem. In other words DO NOT FEEL BAD IF YOU ARE NOT ABLE TO FOLLOW THE NEXT SESSION. Especially if you are a biologist since the folowing presents a certain thought process that takes a lot of time to develop -- time that you may not have put in if you haven't taken a lot of more theoretical classes.


![title](imgs/math_ahead.png)

At this point one idea is to try and model the conditional probability that we observe the current variation in brains given they were drawn from different distributions i.e.

$$
\mathbb{P}(B_1,\ldots, B_n | B_i \sim \mathbb{D} \ \ \forall i \in D, B_i \sim \mathbb{H} \ \ \forall i \in H )
$$

The idea being if this probability is high then we have strong evidence suggesting there is a difference in connectivity levels of depressed and happy brains. However using such a model requires prior knowledge of the distributions $\mathbb{D}$ and $\mathbb{H}$. Knowledge that we do not neccessarily have *a priori*. Keeping this in mind let us consider the second natural approach to try and guage whether or not there is a difference between happy and depressed brains. Namely:

$$
\mathbb{P}(B_1,\ldots, B_n | B_i \sim \mathbb{B} \ \ \forall i ).
$$

This approach seems to be superior to the previous one in the sense that it only deals with one unknown distribution rather than $2$. Furthermore with this framing of the problem we should be able to rank all of the brains from most connected to least connected -- namely $R(B_1), ...,R(B_i)$ and calculate the probability the rankings appear the way they do assuming $B_i \sim \mathbb{B} \ \ \forall i$. More rigorously we will look at

$$
\mathbb{P}(R(B_1),\ldots, R(B_n) | B_i \sim \mathbb{B} \ \ \forall i ).
$$

where $R$ is a function that maps the test statistic to its ranking. This seems to have promise since once we look at the rankings we are somehow overcoming the problem of the unknown distribution. However we are not quite there. In particular in it's hard to see how this calculation bears any practical significance given as n grows very large the probablity that the rankings take any specific form tends towards $0$. Thus all the probabilities are low. One response is to consider the rank-sum of all of the depressed brains and calculate what the probability of seeing a rank-sum as extreme or more extreme then the computed rank-sum. More precisely

$$
\mathbb{P}(\sum_{i \in D} R(B_i) \ | \ B_i \sim \mathbb{B} \ \ \forall i ).
$$

Here we are measuring the probability that a randomly selected subset of size $|D|$ from an original set of size $|B|$ has rank sum as extrme as $sum_{i \in D} R(B_i)$. Indeed, we can calculate the distribution of the rank sum of a subset of size $|D|$ from a superset of size $|S|$ from first principles and calculate $ \mathbb{P}(\sum_{i \in D} R(B_i) \ | \ B_i \sim \mathbb{B} \ \ \forall i )$. This is in fact called the Wilcoxon Rank sum test; and there is code for it in python !

In [ ]:
# Wilcoxon rank sum test
# Reference https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.wilcoxon.html
test_statistic,p_value= stats.ranksums(depressed,happy)
print("-------------------------------------------------------------------------------------------------------------")
print("The probability we observe a rank sum this extreme given the brains were drawn from the same distribution is:")
print(p_value)
print("-------------------------------------------------------------------------------------------------------------")
print("In other words if the brains were drawn from the same distribution we would expect to see a rank sum this extreme")
print(str(p_value*100) + "% of the time")
print("-------------------------------------------------------------------------------------------------------------")

## Concept Check 

In the real world we would just, google it as is done above. But for pedagogical reasons lets see if we can code this up in python. 

Remark: Below I use an iterator (thats what combinations returns).  An iterator is an object in python that CAN be iterated over; but CANNOT be"accessed" like a list. In python this tends to give certain performaince 
benefits. See : https://stackoverflow.com/questions/628903/performance-advantages-to-iterators

If you are interested in learning more about iterators in python. You should read
the wiki on streams: https://en.wikipedia.org/wiki/Stream_(computing) 

In [ ]:
def get_p_val_func_for(num_small ,num_total):
    """
    Returns function that evaluates p-value given sum_statisitc
    from num_small,num_total distribution
    """
    num_sums = int(round(special.binom(33,16)))
    vals = np.zeros(num_sums)
    i = 0
    for combination in combinations(np.arange(num_total),num_small):
        vals[i] = sum(combination)
    vals = np.sort(vals)
    def compute_p_value(obs_sum):
        num_bigger_or_equal_to = 0
        for val in vals:
            if val > obs_sum:
                num_bigger_or_equal_to += 1
            else:
                break
        prop_bigger = num_bigger_or_equal_to / len(vals)
        p_value = 2 * min(prop_bigger,1-prop_bigger)
        return p_value         
    return comput_p_value

def compute_rank_sum_stat(depressed,happy):
    """
    Part 0 Compute CDF for given sample sizes (Assume already have)
    Part 1 get rank sum statistic for depressed
    Part 2 Use CDF to Tell if rank sum computed is "statistically significant"
    """
    concatenated = np.concatenate((depressed, happy), axis=0)
    sorted_lst = np.sort(concatenated)
    def compute_rank_of(x):
        ix = np.isin(sorted_lst,x)
        loc = np.where(ix)
        return loc
    return sum([compute_rank_of(depressed_brain) for depressed_brain in depressed])

def comput_p_value(depressed,happy):
    p_val = get_p_val_func_for(len(depressed),len(happy))
    obs_sum = compute_rank_sum_stat(depressed,happy)
    return p_val(obs_sum)


p_val_func = get_p_val_func_for(3,5) 



In [ ]:
p_val_func

Whew! We did it! [Natural question how do we do this for large sample sizes?] 

[Put picture of asymptotic results asymptopical Normal distribution]

If you know what the two sample non-parametric bootstrap you may find this post interesting 
https://stats.stackexchange.com/questions/61787/can-bootstrap-be-used-to-replace-non-parametric-tests?rq=1

### If you are interested here are some additional articles/sources

https://www.scientificamerican.com/article/brain-imaging-identifies-different-types-of-depression/